In [1]:
import numpy as np
import pandas as pd

In [7]:
csv_file = "train_v2.csv"
df_train = pd.read_csv(csv_file)
csv_file = "test_v2.csv"
df_test = pd.read_csv(csv_file)

In [8]:
df_train.head()

,ID,user,movie,rating
0,610739,3704,3784,3
1,324753,1924,802,3
2,808218,4837,1387,4
3,133808,867,1196,4
4,431858,2631,3072,5


In [38]:
df_train.rename(columns ={'rating': 'Label'}, inplace =True)
df_train.head()

,ID,user,movie,Label
0,610739,3704,3784,3
1,324753,1924,802,3
2,808218,4837,1387,4
3,133808,867,1196,4
4,431858,2631,3072,5


In [32]:
print len(df_train.ID.unique())
print len(df_train.user.unique())
print len(df_train.movie.unique())

750156
6040
3676


In [27]:
# Based on Kaggle kernel by Scirpus
def convert_to_ffm(df,type,numerics,categories,features):
    currentcode = len(numerics)
    catdict = {}
    catcodes = {}
    # Flagging categorical and numerical fields
    for x in numerics:
         catdict[x] = 0
    for x in categories:
         catdict[x] = 1
    
    nrows = df.shape[0]
    ncolumns = len(features)
    with open(str(type) + "_ffm.txt", "w") as text_file:
# Looping over rows to convert each row to libffm format
        for n,r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow['Label']))
             # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(catdict.keys()):
                if(catdict[x]==0):
                    datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
                else:
            # For a new field appearing in a training example
                    if(x not in catcodes):
                        catcodes[x] = {}
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
            # For already encoded fields
                    elif(datarow[x] not in catcodes[x]):
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
                    code = catcodes[x][datarow[x]]
                    datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

In [28]:
num_col = pd.DataFrame()
cat_col = df_train.iloc[:,0:2]
all_col = num_col + cat_col

In [29]:
convert_to_ffm(df_train,'Train',num_col,cat_col,all_col)

In [39]:
import xlearn as xl
ffm_model = xl.create_fm()  # Use field-aware factorization machine
ffm_model.setTrain("Train_ffm.txt")   # Training data
# param:
#  0. binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
param = {'task':'binary', # ‘binary’ for classification, ‘reg’ for Regression
         'k':10,           # Size of latent factor 
         'lr':0.1,        # Learning rate for GD
         'lambda':0.0002, # L2 Regularization Parameter
         'metric':'auc',  # Metric for monitoring validation set performance
         'epoch':25       # Maximum number of Epochs
        }
# Train model

ffm_model.fit(param,'model.out')